In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from colormath.color_objects import sRGBColor, XYZColor
from colormath.color_conversions import convert_color
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
MF = np.genfromtxt("CMF_1nm.csv", delimiter=',') 
print(MF.shape)
CMF = MF[:, 1:4]

(441, 4)


In [3]:
def clip2rgb(xyz):
    XYZ = XYZColor(xyz[0], xyz[1], xyz[2])
    crgb = convert_color(XYZ, sRGBColor)
    r = crgb.rgb_r
    g = crgb.rgb_g
    b = crgb.rgb_b
    r = r if r > 0 else 0
    g = g if g > 0 else 0
    b = b if b > 0 else 0
    r = r if r < 1 else 1
    g = g if g < 1 else 1
    b = b if b < 1 else 1
    return [r, g, b]

In [4]:
def XYZ2xyY(X, Y, Z):
    r = X + Y + Z
    x = X/r
    y = Y/r
    return (x, y, Y)

In [5]:
def g_euc(r1, r2):
    g = 0
    for i in enumerate(r1):
        g+= r1[i]*r2[i]
    return np.sqrt(g)

In [6]:
# plot chromaticity diagram in xyY and lines with white point
L = CMF.shape[0]
abeam = np.ones(L)
fig = plt.figure()
plt.ylim(0,1)
white = np.dot(abeam, CMF)
wtpoint = white/np.linalg.norm(white)
(wx, wy, _) = XYZ2xyY(wtpoint[0], wtpoint[1], wtpoint[2])
plt.scatter(wx, wy, color=(0.9, 0.9, 0.9))
xvec = np.linspace(0, 1, 10)
for i, c in enumerate(CMF):
    rgb = clip2rgb(c)
    xyY = XYZ2xyY(c[0], c[1], c[2])
    m = (wy - xyY[1])/(wx - xyY[0])
    yvec = m*xvec + xyY[1] - m*xyY[0]
    plt.plot(xvec, yvec, color=rgb)
    plt.scatter(xyY[0], xyY[1], color=rgb)
    plt.text(xyY[0], xyY[1], i)

In [8]:
def g_line2point(ln, pt):
    # distance between point and line
    # line ~ (a,b,c) coefficients of ax + by + c = 0
    # point ~ (x0, y0)
    return abs(ln[0]*pt[0] + ln[1]*pt[1] + ln[2])/np.sqrt(ln[0]**2 + ln[1]**2)

In [9]:
def wtangle(p1, p2, wt):
    """Angle between p1 and p2 w.r.t. white point, 
        where all three are given in xy"""
    v1 = np.array([p1[0] - wt[0], p1[1] - wt[1]])
    v2 = np.array([p2[0] - wt[0], p2[1] - wt[1]])
    return np.dot(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))

In [10]:
def points2line(p1, p2):
    # return (a,b,c) of equation of lines for points
    # in form ax + by + c = 0
    b = 1
    slope = (p1[1] - p2[1])/(p1[0] - p2[0])
    a = -slope
    c = slope*p1[0] - p1[1]
    return (a,b,c)

# algorithm to find complementary wavelength

1) determine complementary wavelength of the last point in the cmf $\bar{\lambda}_f$ by searching first half of the CMF, call this $\lambda_f$. 

2) Call $\lambda_0$ the first point in the CMF, and find its complementary wavelength $\bar{\lambda}_0$

2) If input wavelength is in $[\lambda_0, \lambda_f]$ or $[\bar{\lambda_0}, \bar{\lambda_f}]$, then find its complementary wavelength. Otherwise, return -1

In [11]:
def compl(wl, abeam, cmf):
    """Make pairs of complementary wavelengths"""
    L = cmf.shape[0]
    cwl = CMF[wl]
    xywl = XYZ2xyY(cwl[0], cwl[1], cwl[2])[0:2]
    wt = np.dot(abeam, cmf)
    xywt = XYZ2xyY(wt[0], wt[1], wt[2])[0:2]
    
    # [1] find complementary wavelength of last element of cmf
    xyf = XYZ2xyY(cmf[-1][0], cmf[-1][1], cmf[-1][2])[0:2]
    ang = 0
    lamf = 0
    for i, c in enumerate(cmf):
        xyc = XYZ2xyY(c[0], c[1], c[2])[0:2]
        angp = abs(wtangle(xyf, xyc, xywt))
        if angp > ang:
            ang = angp 
        else: 
            lamf = i-1
            break
    
    
    # [2] find complementary wavelength of cmf[0]
    xy0 = XYZ2xyY(cmf[0][0], cmf[0][1], cmf[0][2])[0:2]
    l0 = points2line(xy0, xywt)
    distset = []
    for i, c in enumerate(cmf[lamf:]):
        xyc = XYZ2xyY(c[0], c[1], c[2])[0:2]
        distset.append(g_line2point(l0, xyc))
    comp0 = np.argmin(distset)+lamf
    
    # [3] check ranges and find complemtary wavelengths
    compout = -1
    distset = []
    ln = points2line(xywl, xywt)
    if (wl >= 0) and (wl <= lamf):
        # in first range
        for c in cmf[comp0:]:
            xyc = XYZ2xyY(c[0], c[1], c[2])[0:2]
            distset.append(g_line2point(ln, xyc))
        compout = np.argmin(distset)+comp0
    elif (wl >= comp0) and (wl <= L):
        # in second range
        for c in cmf[0:(lamf+1)]:
            xyc = XYZ2xyY(c[0], c[1], c[2])[0:2]
            distset.append(g_line2point(ln, xyc))
        compout = np.argmin(distset)
    return compout

In [12]:
# print all complementaries
L = CMF.shape[0]
print(L)
abeam = np.ones(L)
for i in range(L):
    print(compl(i, abeam, CMF))

441
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
182
182
182
182
182
182
182
182
183
183
183
183
183
183
184
184
184
184
185
185
186
186
187
187
188
188
189
190
191
192
193
194
196
198
200
203
206
210
215
223
236
313
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
22
51
64
71
75
78
81
83
84
86
87
88
89
90
91
91
92
92
93
93
94
94
95
95
95
96
96
96
97
97
97
97
97
98
98
98
98
98
98
99
99
99
99
99
99
99
99
99
99
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101

In [13]:
def bump(I, x0, xf):
    out = np.zeros(len(I))
    if x0 < xf:
        out[x0:xf] = 1
    elif x0 > xf:
        out[x0:] = 1
        out[0:xf] = 1
    return out

In [14]:
def Optimal(wl1, wl2, cmf, step=1):
    """
    Optimal windowing function with wrapping
    """
    Norm = np.sum(CMF, 0)
    X =  np.dot(cmf[:,0], bump(cmf, wl1, wl2))
    Y =  np.dot(cmf[:,1], bump(cmf, wl1, wl2))
    Z =  np.dot(cmf[:,2], bump(cmf, wl1, wl2))
    XYZ = [X, Y, Z]
    return (XYZ[0]/Norm[0], XYZ[1]/Norm[1], XYZ[2]/Norm[2])

In [15]:
# graph semichromes
L = CMF.shape[0]
abeam = np.ones(L)
fig = plt.figure()
white = np.dot(abeam, CMF)
wtpoint = white/np.linalg.norm(white)
plt.text(0, -1, "gaps")
plt.text(0, 1, "bumps")
for i in range(L):
    c = compl(i, abeam, CMF)
    if c != -1:
        oc = Optimal(i, c, CMF)
        rgb = clip2rgb(oc)
        plt.scatter(i, +1, color=rgb)
        plt.text(i, 0.05*(-1)**i, f'{c}')
        oc = Optimal(i, c, CMF)
        rgb = clip2rgb(oc)
        plt.scatter(i, -1, color=rgb)

In [24]:
# plot semichrome optimal functions
L = CMF.shape[0]
abeam = np.ones(L)
fig = plt.figure()
white = np.dot(abeam, CMF)
xrange = np.linspace(0, L, 100)
plt.plot(CMF, color=(0,0,0))
for i in range(L):
    c = compl(i, abeam, CMF)
    if c != -1:
        oc = Optimal(i, c, CMF)
        print(i, c, oc)
        rgb = clip2rgb(oc)
        plt.plot(xrange, bump(xrange, c, i), color=rgb)

        

0 36 (0.3224975987484593, 0.5700139703742155, 0.9999357221521815)
1 36 (0.32233087201636906, 0.5699956315176254, 0.9991188803539006)
2 36 (0.3219158752208547, 0.5699487623872896, 0.9970804752283385)
3 36 (0.320936517375622, 0.5698402995608411, 0.992233663590142)
4 36 (0.3188387544919101, 0.569620395883513, 0.9817570703482087)
5 36 (0.31487860036084836, 0.5692187852118626, 0.9618214671235786)
6 36 (0.3084821782522768, 0.5685865584407325, 0.9291929782715853)
7 36 (0.2994784056552337, 0.5676898342337812, 0.8826823973512264)
8 36 (0.28847198230786686, 0.5665344754010659, 0.8249761144516499)
9 36 (0.275564948333192, 0.5650664339550384, 0.7563115419357589)
10 36 (0.26129194957015717, 0.5632273383345316, 0.6790041689022817)
11 36 (0.24588905248860893, 0.561000904921872, 0.5942149678950869)
12 36 (0.23076953335867667, 0.5584605486605626, 0.5093834049874149)
13 36 (0.21650737063661912, 0.5555977670385145, 0.42764240361225264)
14 37 (0.23924582517243848, 0.5954444279108025, 0.35406651447052306)


In [ ]:
# now plot complemtary lines colored with optimal color 
L = CMF.shape[0]
abeam = np.ones(L)
fig = plt.figure()
plt.ylim(0,1)
white = np.dot(abeam, CMF)
wtpoint = white/np.linalg.norm(white)
(wx, wy, _) = XYZ2xyY(wtpoint[0], wtpoint[1], wtpoint[2])
plt.scatter(wx, wy, color=(0.9, 0.9, 0.9))
xvec = np.linspace(0, 1, 10)
for i, c in enumerate(CMF):
    rgb = clip2rgb(c)
    xyY = XYZ2xyY(c[0], c[1], c[2])
    m = (wy - xyY[1])/(wx - xyY[0])
    yvec = m*xvec + xyY[1] - m*xyY[0]
    plt.plot(xvec, yvec, color=rgb)
    plt.scatter(xyY[0], xyY[1], color=rgb)
    plt.text(xyY[0], xyY[1], i)

In [19]:
O = Optimal(20, 47, CMF)
print(clip2rgb(O))

[0.8173369609366901, 1, 0]


---
# end of investigation
---
old stuff that didn't work follows

In [174]:
def comp(wl, abeam, cmf):
    # find closest point to to the line defined
    # by the wl and white point in the chromaticity diagram 
    xyz = cmf[wl]
    wt = np.dot(abeam, cmf)
    slope = (wt[1]-xyz[1])/(wt[0]-xyz[0])
    a = -slope
    b = 1
    c = slope*wt[0]-wt[1]
    line = (a,b,c)
    
    
    distset = [g_line2point(line, cm) for cm in cmf]
    idx = np.argmin(distset)
    return (idx, distset[idx])

In [89]:
# plot complementary wavelengths
L = CMF.shape[0]
abeam = np.ones(L)
for i in range(L):
    print(comp(i, abeam, CMF))

(0, 3.5018691428121256e-16)
(1, 3.752904265821472e-16)
(2, 2.1628944139170706e-15)
(3, 1.2157074835948988e-15)
(4, 2.017936397908236e-15)
(5, 2.544073733007683e-16)
(6, 3.322910319328499e-16)
(7, 1.9333852830910202e-15)
(8, 4.292916534679e-16)
(9, 9.361563142553548e-16)
(10, 5.458813742401501e-16)
(11, 1.950171281576272e-16)
(12, 7.024857689067976e-16)
(13, 1.3283421321511862e-15)
(14, 4.1065668686695703e-16)
(15, 1.6345861769380442e-15)
(16, 3.273940607947942e-15)
(17, 1.001500361177521e-15)
(18, 3.982958913847548e-15)
(19, 2.363587171340332e-16)
(20, 3.549811130788198e-16)
(21, 1.2243571999595549e-15)
(22, 4.746565516141783e-16)
(23, 0.0)
(24, 3.17554538831979e-16)
(25, 1.4313131285297986e-15)
(26, 1.0351471253211816e-15)
(27, 3.187240933971699e-16)
(28, 1.9924524008951436e-15)
(29, 7.968496966838487e-17)
(30, 1.7522873240431849e-15)
(31, 3.18299665543048e-16)
(32, 1.2714906987248266e-15)
(33, 7.141417635847118e-16)
(34, 7.127438510982076e-16)
(35, 1.6595328922122784e-15)
(36, 1.2611

In [90]:
# plot complementary pairs in chromaticity diagram
L = CMF.shape[0]
abeam = np.ones(L)
fig = plt.figure()
white = np.dot(abeam, CMF)
wtpoint = white/np.linalg.norm(white)
(wx, wy, _) = XYZ2xyY(wtpoint[0], wtpoint[1], wtpoint[2])
plt.scatter(wx, wy, color=(0.9, 0.9, 0.9))
for i, c in enumerate(CMF):
    rgb = clip2rgb(c)
    p1 = XYZ2xyY(c[0], c[1], c[2])
    (wl2, _) = comp(i, abeam, CMF)
    c2 = CMF[wl2]
    p2 = XYZ2xyY(c2[0], c2[1], c2[2])
    plt.plot((p1[0], p2[0]), (p1[1], p2[1]), color=rgb)

The plane containing $w, k, c$ can be defined with a point and normal vector $p, n$. The normal vector can be determined via the cross product $n = (w-k) \times (c-k)$. The distance between another point $p$ and the plane is then
$$
    D = n \cdot p
$$
which comes from a change of coordinants of the Hesse normal form using the fact that $k = 0$ and the plane intersects the origin
https://en.wikipedia.org/wiki/Plane_(geometry)

In [5]:
def pnormal(w, c):
    # unit normal vector of a plane
    # assumes black located at origin
    n = np.cross(w, c)
    return (1/np.linalg.norm(n))*n

In [6]:
def detdist(p, w, c):
    # determinnant assuing 1 point (black) at origin
    a = np.array(
        [p-w, 
        p-c, 
        p
        ]
    )
    return np.linalg.det(a)

In [7]:
def distP2P(pt, n):
    # returns distance between a point and a plane defined by unit normal
    return np.dot(pt, n)

In [10]:
def compl(wl, abeam, cmf):
    # calculate complementary wavelength using by finding
    # element in cmf to solve 
    # cmf[wl] + e = abeam*cmf

In [11]:
def compset(abeam, cmf, eps=0.1):
    # calculate set of complementary wavelengths using normal vectors
    white = np.dot(abeam, cmf)
    setout = []
    for i1, c1 in enumerate(cmf):
        p1 = pnormal(white, c1)
        for i2, c2 in enumerate(cmf[::-1]):
            p2 = pnormal(white, c2)
            if np.dot(p1, p2) > 1-eps:
                setout.append((i1, i2))
                break
    return setout

In [12]:
abeam = np.ones(L)
print(compset(abeam, CMF))

NameError: name 'L' is not defined

In [13]:
def compl3(wl, abeam, cmf):
    # returns complementary wavelength associated with 
    # wavelength "wl" under achromatic beam "abeam"
    # assumes "index representation" of wavelengths
    xyz = cmf[wl]
    white = np.dot(abeam, cmf)
    pl = pnormal(white, xyz)
    distset = [distP2P(c, pl) for c in cmf]
    idx = np.argmin(distset)
    return (idx, distset[idx])

In [14]:
def compl2(wl, abeam, cmf):
    # use of determinnat distance 
    xyz = cmf[wl]
    white = np.dot(abeam, cmf)
    distset = []
    for idx, c in enumerate(cmf):
        if idx != wl:
            distset.append(detdist(c, white, xyz))
    idx = np.argmin(distset)
    return (idx, distset[idx])

In [43]:
# plot planes 
L = CMF.shape[0]
abeam = np.ones(L)
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_zlim(-1,1)
white = np.dot(abeam, CMF)
wtpoint = white/np.linalg.norm(white)
print(wtpoint)
for i in range(L):
    p1 = CMF[i]
    x = np.linspace(p1[0],wtpoint[0]*0.9,10)
    y = np.linspace(p1[1],wtpoint[1]*0.9,10)

    X,Y = np.meshgrid(x,y)
    n = pnormal(white, p1)
    if abs(n[2]) > 0:
        Z=-(n[0]*X + n[1]*Y)/n[2]
    
    rgb = clip2rgb(p1)
    if sum(rgb) > 0.1:
        rgba = rgb+[0.1]
        ax.plot_surface(X,Y,Z, color=rgba)
    ax.scatter(p1[0], p1[1], p1[2], color=rgb) 
    ax.plot((0,wtpoint[0]), (0,wtpoint[1]), (0,wtpoint[2]), color=(1,1,1))

[0.57731787 0.57728924 0.57744368]


NameError: name 'pnormal' is not defined

# calculate complementaries by minimum angle between their normals 

Print the complementary wavelengths. Notice the gap of greenss that have no complementary wavelength

In [21]:
# ordinary distance
L = CMF.shape[0]
abeam = np.ones(L)
fig = plt.figure()
ax = fig.gca(projection='3d')
white = np.dot(abeam, CMF)
wtpoint = white/np.linalg.norm(white)
for i in range(L):
    C = compl(i, abeam, CMF)
    print(C)
    rgb = clip2rgb(CMF[i])
    p1 = CMF[i]
    p2 = CMF[C[0]]
    ax.scatter(p1[0], p1[1], p1[2], color=rgb) 
    ax.plot((0,wtpoint[0]), (0,wtpoint[1]), (0,wtpoint[2]), color=(1,1,1))
    ax.plot((p1[0], p2[0]), (p1[1], p2[1]), (p1[2], p2[2]), color=rgb)

(27, -0.4045934045270105)
(27, -0.40501857559953924)
(27, -0.40546987309516586)
(27, -0.4057752509988971)
(27, -0.4061675319383515)
(27, -0.40703175468245123)
(27, -0.40806328959168703)
(27, -0.40956188293950135)
(27, -0.41123273349547845)
(27, -0.41352013022937123)
(27, -0.41565693052165986)
(27, -0.4184131523760544)
(27, -0.42178258297052296)
(27, -0.4266993984696308)
(27, -0.43377978401153383)
(28, -0.44433847568805435)
(28, -0.4612490958436624)
(28, -0.4876507616839592)
(29, -0.5279207758090011)
(30, -0.5887995737013016)
(30, -0.6616028502913694)
(33, -0.7014763440186021)
(37, -0.7386664105917673)
(40, -0.8038765171359037)
(41, -0.8223293756480541)
(42, -0.8182745672071307)
(42, -0.8106498527454721)
(42, -0.7989416914601637)
(42, -0.7840820496906293)
(43, -0.7666810412137525)
(43, -0.7475762627692393)
(43, -0.7235846859774734)
(43, -0.6939283165387471)
(43, -0.6563050816707104)
(43, -0.6064685052566009)
(43, -0.548258355529379)
(44, -0.480360889438527)
(44, -0.4073285771425659)
(44

In [38]:
print(sum(CMF))

[22.60973362 22.60861237 22.61466056]


In [ ]:
L = CMF.shape[0]
abeam = np.ones(L)
fig = plt.figure()
ax = fig.gca(projection='3d')
white = np.dot(abeam, CMF)
for i in range(L):
    C = compl2(i, abeam, CMF)
    print(C)
    rgb = clip2rgb(CMF[i])
    p1 = CMF[i]
    p2 = CMF[C[0]]
    ax.scatter(p1[0], p1[1], p1[2], color=rgb) 
    ax.plot((p1[0], p2[0]), (p1[1], p2[1]), (p1[2], p2[2]), color=rgb) 